#drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')
#ai lab jp

Mounted at /content/drive


#geodesc.git

#contextdesc

## 環境構築

In [ ]:
!pip uninstall -y tensorflow-gpu
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0
!pip install tensorflow==1.14.0
!pip uninstall  -y opencv-contrib-python
!pip uninstall -y  opencv-python
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17
!pip install open3d==0.10
# !rm -rf /usr/local/lib/python3.6/dist-packages/cv2

In [20]:
# !pip uninstall -y open3d-python

## ソースダウンロード

/content/drive/MyDrive/00_work/004_pose_estimation


In [ ]:
# !git clone https://github.com/lzx551402/contextdesc.git

##訓練済みモデルダウンロード

In [ ]:
# !mkdir {bp}/contextdesc/pretrained
# %cd {bp}/contextdesc/pretrained
# !wget https://research.altizure.com/data/contextdesc_models/contextdesc_pp.tar
# !tar -xvf contextdesc_pp.tar
# !wget https://research.altizure.com/data/contextdesc_models/retrieval_model.tar
# !tar -xvf retrieval_model.tar
# !wget https://research.altizure.com/data/contextdesc_models/dense-contextdesc.tar
# !tar -xvf dense-contextdesc.tar

In [6]:

bp='/content/drive/MyDrive/00_work/004_pose_estimation/'
%cd {bp}
%cd {bp}/contextdesc
# !python image_matching.py 
  # --loc_model '/content/contextdesc/pretrained/dense-contextdesc'
# --reg_model 'pretrained/contextdesc++'

/content/drive/MyDrive/00_work/004_pose_estimation
/content/drive/MyDrive/00_work/004_pose_estimation/contextdesc


## 特徴取得

In [ ]:
#!/usr/bin/env python3
import sys
%cd {bp}/contextdesc
LOCAL_PATH = '../'
if LOCAL_PATH not in sys.path:
    sys.path.append(LOCAL_PATH)
import os
! echo $PYTHONPATH
os.environ['PYTHONPATH'] = '/env/python:{bp}/contextdesc/'
from models import get_model
from models.reg_model import RegModel
UTILS_PATH="{bp}/contextdesc/"
if UTILS_PATH not in sys.path:
    sys.path.append(UTILS_PATH)
# get_model('reg_model')
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

UTILS_PATH = 'utils'
if UTILS_PATH not in sys.path:
    sys.path.append(UTILS_PATH)
from opencvhelper import MatcherWrapper

from easydict import EasyDict

UTILS_PATH = '../models'
if UTILS_PATH not in sys.path:
    sys.path.append(UTILS_PATH)
from models import get_model



FLAGS = EasyDict({})
FLAGS.loc_model='pretrained/contextdesc++'
FLAGS.reg_model='pretrained/retrieval_model'
FLAGS.img1_path='imgs/rgb_cam0.png'
FLAGS.img2_path='imgs/rgb_cam1.png'
FLAGS.n_sample=2048
FLAGS.model_type='pb'
FLAGS.dense_desc=False
FLAGS.ratio_test=False
FLAGS.cross_check=False


def load_imgs(img_paths):
    rgb_list = []
    gray_list = []
    for img_path in img_paths:
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)[..., np.newaxis]
        img = img[..., ::-1]
        rgb_list.append(img)
        gray_list.append(gray)
    return rgb_list, gray_list


def extract_regional_features(rgb_list, model_path):
    reg_feat_list = []
    model = get_model('reg_model')(model_path)
    for _, val in enumerate(rgb_list):
        reg_feat = model.run_test_data(val)
        reg_feat_list.append(reg_feat)
    model.close()
    return reg_feat_list


def extract_local_features(gray_list, model_path):
    cv_kpts_list = []
    loc_info_list = []
    loc_feat_list = []
    sift_feat_list = []
    model = get_model('loc_model')(model_path, **{'sift_desc': True,
                                                  'n_sample': FLAGS.n_sample,
                                                  'peak_thld': 0.04,
                                                  'dense_desc': FLAGS.dense_desc,
                                                  'upright': False})
    for _, val in enumerate(gray_list):
        loc_feat, kpt_mb, normalized_xy, cv_kpts, sift_desc = model.run_test_data(val)
        raw_kpts = [np.array((i.pt[0], i.pt[1], i.size, i.angle, i.response)) for i in cv_kpts]
        raw_kpts = np.stack(raw_kpts, axis=0)
        loc_info = np.concatenate((raw_kpts, normalized_xy, loc_feat, kpt_mb), axis=-1)
        cv_kpts_list.append(cv_kpts)
        loc_info_list.append(loc_info)
        sift_feat_list.append(sift_desc)
        loc_feat_list.append(loc_feat / np.linalg.norm(loc_feat, axis=-1, keepdims=True))
    model.close()
    return cv_kpts_list, loc_info_list, loc_feat_list, sift_feat_list


def extract_augmented_features(reg_feat_list, loc_info_list, model_path):
    aug_feat_list = []
    model = get_model('aug_model')(model_path, **{'quantz': False})
    assert len(reg_feat_list) == len(loc_info_list)
    for idx, _ in enumerate(reg_feat_list):
        aug_feat, _ = model.run_test_data([reg_feat_list[idx], loc_info_list[idx]])
        aug_feat_list.append(aug_feat)
    model.close()
    return aug_feat_list

In [8]:
cam_zed2_22378008_left_hd1080 = EasyDict({})
cam_zed2_22378008_left_hd1080.fx=1055.26
cam_zed2_22378008_left_hd1080.fy=1054.92
cam_zed2_22378008_left_hd1080.cx=962.91
cam_zed2_22378008_left_hd1080.cy=567.182
cam_zed2_22378008_left_hd1080.depthScale=1000.0

'''

      "fx": 1058.51,
      "fy": 1057.56,
      "cx": 972.89,
      "cy": 570.658,
'''
cam_zed2_21888201_left_hd1080 = EasyDict({})
cam_zed2_21888201_left_hd1080.fx=1058.51
cam_zed2_21888201_left_hd1080.fy=1057.56
cam_zed2_21888201_left_hd1080.cx=972.89
cam_zed2_21888201_left_hd1080.cy=570.658
cam_zed2_21888201_left_hd1080.depthScale=1000.0

'''

   "StereoLabs_ZED2_22115402_LEFT_HD1080": {
      "fx": 1058.82,
      "fy": 1058.16,
      "cx": 909.73,
      "cy": 560.099,
'''
cam_zed2_22115402_left_hd1080 = EasyDict({})
cam_zed2_22115402_left_hd1080.fx=1058.82
cam_zed2_22115402_left_hd1080.fy=1058.16
cam_zed2_22115402_left_hd1080.cx=909.73
cam_zed2_22115402_left_hd1080.cy=560.099
cam_zed2_22115402_left_hd1080.depthScale=1000.0

bp='/content/drive/MyDrive/00_work/004_pose_estimation/contextdesc/imgs/'

imgdic = EasyDict({})
imgdic.rgb0=cv2.imread(f"{bp}/rgb_cam0.png")
imgdic.rgb1=cv2.imread(f"{bp}/rgb_cam1.png")
imgdic.rgb2=cv2.imread(f"{bp}/rgb_cam2.png")
imgdic.dpt0=np.load(f"{bp}/depth_cam0.npy")
imgdic.dpt1=np.load(f"{bp}/depth_cam1.npy")
imgdic.dpt2=np.load(f"{bp}/depth_cam2.npy")
imgdic.pcd0=np.load(f"{bp}/pcd_cam0.npy")
imgdic.pcd1=np.load(f"{bp}/pcd_cam1.npy")
imgdic.pcd2=np.load(f"{bp}/pcd_cam2.npy")
'''
cam_zed2_21888201_left_hd1080:cam0
cam_zed2_22378008_left_hd1080:cam1
cam_zed2_22115402_left_hd1080:cam2
get_good_matches_pts(goodMatches,kp1,kp2,imgdic,cams_para)
'''
cams_para=[cam_zed2_21888201_left_hd1080,cam_zed2_22378008_left_hd1080,cam_zed2_22115402_left_hd1080]
def get_good_matches_pts(goodMatches,kp1,kp2,imgdic,cams):
    pts_obj1=[]
    pts_obj2=[]
    pts_img1=[]
    pts_img2=[]
    def make_obj(p,d,cam):
        pt=[p[1],p[0],d]
        pd= point2dTo3d(pt,cam)
        return pd
    for i, goodMatch in enumerate(goodMatches):
        p1=np.array(kp1[goodMatch.queryIdx].pt,dtype=np.int)
        p2=np.array(kp2[goodMatch.trainIdx].pt,dtype=np.int)
        d1=imgdic.dpt0[p1[1],p1[0]]
        d2=imgdic.dpt1[p2[1],p2[0]]
        if d1 == 0 or d2 == 0 :
            continue;
        pts1=make_obj(p1, d1,cams[0])
        pts2=make_obj(p2,d2,cams[1])
        if np.isnan(np.mean(pts1))  or np.isnan(np.mean(pts2)) or np.isinf(np.mean(pts1)) or np.isinf(np.mean(pts2)):
            continue
        pts_obj1.append(pts1)
        pts_obj2.append(pts2)
        pts_img1.append(p1)
        pts_img2.append(p2)

    if len(pts_obj1)==0 or len(pts_img2)==0:
        return -1
    print(f"pts_obj1:{len(pts_obj1)},pts_obj2:{len(pts_obj2)}")
    id=0
    cameraMatrix1=[
        [cams[id].fx,0,cams[id].cx],
        [0,cams[id].fy,cams[id].cy],
        [0,0,1]
        ]
    id=1
    cameraMatrix2=[
        [cams[id].fx,0,cams[id].cx],
        [0,cams[id].fy,cams[id].cy],
        [0,0,1]
        ]
    distCoeffs=[0, 0, 0, 0, 0]
    pts_img1=np.array(pts_img1,dtype=np.float32)
    pts_img2=np.array(pts_img2,dtype=np.float32)
    pts_obj1 = np.array(pts_obj1, dtype=np.float32)
    pts_obj2 = np.array(pts_obj2, dtype=np.float32)
    cameraMatrix1 = np.array(cameraMatrix1, dtype=np.float64)
    cameraMatrix2 = np.array(cameraMatrix2, dtype=np.float64)
    distCoeffs = np.array(distCoeffs, dtype=np.float64)

    _, rVec1, tVec1=cv2.solvePnP(pts_obj1, pts_img1, cameraMatrix1, distCoeffs=distCoeffs)
    _, rVec2, tVec2=cv2.solvePnP(pts_obj2, pts_img2, cameraMatrix2, distCoeffs=distCoeffs)
    R1=cv2.Rodrigues(rVec1)[0]
    R2=cv2.Rodrigues(rVec2)[0]
    R_12 = R2 @ R1.T
    # t_12= R2 @ (-R1.T @ tVec1) + tVec2
    t_12= tVec2 - R_12@tVec1
    # trans12=generateTransMatrixRt(t_12,R_12)
    trans12 = np.eye(4)
    trans12[:3, :3] = R_12
    trans12[:3, 3] = t_12.flatten()
    return trans12,pts_obj1,pts_obj2

def get_key_points(goodMatches,kp1,kp2,imgdic):
    pts_obj1=[]
    pts_obj2=[]
    for i, goodMatch in enumerate(goodMatches):
        p1=np.array(kp1[goodMatch.queryIdx].pt,dtype=np.int)
        p2=np.array(kp2[goodMatch.trainIdx].pt,dtype=np.int)
        pts1=imgdic.pcd0[p1[1],p1[0]]
        pts2=imgdic.pcd1[p2[1],p2[0]]
        if np.isnan(np.mean(pts1))  or np.isnan(np.mean(pts2)) or np.isinf(np.mean(pts1)) or np.isinf(np.mean(pts2)):
            continue
        pts_obj1.append(pts1[:3])
        pts_obj2.append(pts2[:3])
    return np.array(pts_obj1),np.array(pts_obj2)
def get_key_ply(goodMatches,kp1,kp2,imgdic):
    pts_obj1=[]
    pts_obj2=[]
    cols1=[]
    cols2=[]
    def get_colors(zed_colors,x,y):
        tmp = zed_depthfloat_to_abgr(zed_colors[x, y])
        tmp = [tmp[3], tmp[2], tmp[1]]  # ABGR to RGB
        tmp = np.array(tmp).astype(np.float64) / 255.  # for ply (color is double)
        return tmp
    for i, goodMatch in enumerate(goodMatches):
        p1=np.array(kp1[goodMatch.queryIdx].pt,dtype=np.int)
        p2=np.array(kp2[goodMatch.trainIdx].pt,dtype=np.int)
        pts1=imgdic.pcd0[p1[1],p1[0]]
        pts2=imgdic.pcd1[p2[1],p2[0]]
        if np.isnan(np.mean(pts1))  or np.isnan(np.mean(pts2)) or np.isinf(np.mean(pts1)) or np.isinf(np.mean(pts2)):
            continue
        cols1.append(get_colors(imgdic.pcd0[:, :, 3],p1[1],p1[0]))
        cols2.append(get_colors(imgdic.pcd1[:, :, 3],p2[1],p2[0]))
        pts_obj1.append(pts1[:3])
        pts_obj2.append(pts2[:3])
    ply1=make_pcd(np.array(pts_obj1),np.array(cols1))
    ply2=make_pcd(np.array(pts_obj2),np.array(cols2))
    return ply1,ply2



##main 関数

In [ ]:
if FLAGS.model_type == 'pb':
    reg_model_path = os.path.join(FLAGS.reg_model, 'reg.pb')
    loc_model_path = os.path.join(FLAGS.loc_model, 'loc.pb')
    aug_model_path = os.path.join(FLAGS.loc_model, 'aug.pb')
elif FLAGS.model_type == 'ckpt':
    reg_model_path = os.path.join(FLAGS.reg_model, 'model.ckpt-550000')
    loc_model_path = os.path.join(FLAGS.loc_model, 'model.ckpt-400000')
    aug_model_path = os.path.join(FLAGS.loc_model, 'model.ckpt-400000')
else:
    raise NotImplementedError

img_paths = [FLAGS.img1_path, FLAGS.img2_path]
# load testing images.
rgb_list, gray_list = load_imgs(img_paths)
# extract regional features.
reg_feat_list = extract_regional_features(rgb_list, reg_model_path)
# extract local features and keypoint matchability.
cv_kpts_list, loc_info_list, loc_feat_list, sift_feat_list = extract_local_features(
    gray_list, loc_model_path)
# extract augmented features.
aug_feat_list = extract_augmented_features(reg_feat_list, loc_info_list, aug_model_path)
# feature matching and draw matches.
matcher = MatcherWrapper()
sift_match, sift_mask = matcher.get_matches(
    sift_feat_list[0], sift_feat_list[1], cv_kpts_list[0], cv_kpts_list[1],
    ratio=0.8 if FLAGS.ratio_test else None, cross_check=FLAGS.cross_check,
    err_thld=3, ransac=True, info='SIFT feautre')

base_match, base_mask = matcher.get_matches(
    loc_feat_list[0], loc_feat_list[1], cv_kpts_list[0], cv_kpts_list[1],
    ratio=0.89 if FLAGS.ratio_test else None, cross_check=FLAGS.cross_check,
    err_thld=3, ransac=True, info='Raw local feature')

aug_match, aug_mask = matcher.get_matches(
    aug_feat_list[0], aug_feat_list[1], cv_kpts_list[0], cv_kpts_list[1],
    ratio=0.89 if FLAGS.ratio_test else None, cross_check=FLAGS.cross_check,
    err_thld=3, ransac=True, info='Augmented local feature')

sift_disp = matcher.draw_matches(
    rgb_list[0], cv_kpts_list[0], rgb_list[1], cv_kpts_list[1], sift_match, sift_mask)
base_disp = matcher.draw_matches(
    rgb_list[0], cv_kpts_list[0], rgb_list[1], cv_kpts_list[1], base_match, base_mask)
aug_disp = matcher.draw_matches(
    rgb_list[0], cv_kpts_list[0], rgb_list[1], cv_kpts_list[1], aug_match, aug_mask)

rows, cols = sift_disp.shape[0:2]
white = (np.ones((int(rows / 50), cols, 3)) * 255).astype(np.uint8)
disp = np.concatenate([sift_disp, white, base_disp, white, aug_disp], axis=0)
cv2.imwrite("rgb01.png",disp)
plt.xticks([])
plt.yticks([])
plt.imshow(disp)
plt.show()

In [ ]:
cv2.imwrite("{bp}rgb01.png",disp)

True

In [10]:
def point2dTo3d(point,cam):
    z=point[2]/cam.depthScale
    x=(point[0]-cam.cx)*z/cam.fx
    y=(point[1]-cam.cy)*z/cam.fy
    return [x,y,z]

In [11]:
import numpy as np
def make_pcd(points, colors):
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(points)
    pcd.colors = open3d.utility.Vector3dVector(colors)
    return pcd
def make_merge(pcds):
  points=[]
  colors=[]
  for pcd in pcds:
     points.extend(np.array(pcd.points))
     colors.extend(np.array(pcd.colors))
  return make_pcd(points, colors)

In [12]:
import open3d
import copy


cams_para=[cam_zed2_21888201_left_hd1080,cam_zed2_22378008_left_hd1080,cam_zed2_22115402_left_hd1080]
trans_sift,pts_obj1_sift,pts_obj2_sift=get_good_matches_pts(sift_match,cv_kpts_list[0],cv_kpts_list[1],imgdic,cams_para)
trans_base,pts_obj1_base,pts_obj2_base=get_good_matches_pts(base_match,cv_kpts_list[0],cv_kpts_list[1],imgdic,cams_para)
trans_aug,pts_obj1_aug,pts_obj2_aug=get_good_matches_pts(aug_match,cv_kpts_list[0],cv_kpts_list[1],imgdic,cams_para)


cloud0=open3d.io.read_point_cloud(f'{bp}/pcd_mask_cam0.ply')
cloud1=open3d.io.read_point_cloud(f'{bp}/pcd_mask_cam1.ply')

ext='sift'
cloud0_t=copy.deepcopy(cloud0)
cloud0_gm=cloud0_t.transform(trans_sift)
open3d.io.write_point_cloud(f'{bp}{ext}_pcd_trans_0_solvePnP.ply', cloud0_gm)
ply_merge=make_merge([cloud0_gm,cloud1])
open3d.io.write_point_cloud(f'{bp}{ext}_pcd_merge_solvePnP.ply', ply_merge)


ext='base'
cloud0_t=copy.deepcopy(cloud0)
cloud0_gm=cloud0_t.transform(trans_base)
open3d.io.write_point_cloud(f'{bp}{ext}_pcd_trans_0_solvePnP.ply', cloud0_gm)
ply_merge=make_merge([cloud0_gm,cloud1])
open3d.io.write_point_cloud(f'{bp}{ext}_pcd_merge_solvePnP.ply', ply_merge)


ext='aug'
cloud0_t=copy.deepcopy(cloud0)
cloud0_gm=cloud0_t.transform(trans_aug)
open3d.io.write_point_cloud(f'{bp}{ext}_pcd_trans_0_solvePnP.ply', cloud0_gm)
ply_merge=make_merge([cloud0_gm,cloud1])
open3d.io.write_point_cloud(f'{bp}{ext}_pcd_merge_solvePnP.ply', ply_merge)

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:160: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


pts_obj1:3990,pts_obj2:3990
pts_obj1:3994,pts_obj2:3994
pts_obj1:4011,pts_obj2:4011


True

In [ ]:
# cams_para=[cam_zed2_21888201_left_hd1080,cam_zed2_22378008_left_hd1080,cam_zed2_22115402_left_hd1080]
# # cams_para=[cam_zed2_21888201_left_hd1080,cam_zed2_21888201_left_hd1080,cam_zed2_22115402_left_hd1080]
# trans12,pts_obj1,pts_obj2=get_good_matches_pts(base_match,cv_kpts_list[0],cv_kpts_list[1],imgdic,cams_para)

In [ ]:
# import open3d
# import copy
# cloud0=open3d.io.read_point_cloud(f'{bp}/pcd_mask_cam0.ply')
# cloud1=open3d.io.read_point_cloud(f'{bp}/pcd_mask_cam1.ply')
# # cloud0_t=copy.deepcopy(cloud1)
# cloud0_gm=cloud0.transform(trans12)
# ext='sift'
# open3d.io.write_point_cloud(f'{bp}{ext}_pcd_trans_0_solvePnP.ply', cloud0_gm)


True

In [ ]:
# https://github.com/lzx551402/contextdesc
# https://github.com/neka-nat/probreg

#probreg

##install

In [ ]:
# https://github.com/neka-nat/probreg
!pip install cupy
!pip install probreg
# !pip install open3d==0.10

In [4]:
%cd /content/drive/MyDrive/00_work/004_pose_estimation
# !git clone https://github.com/neka-nat/probreg.git --recursive

[Errno 2] No such file or directory: '/content/drive/MyDrive/00_work/004_pose_estimation'
/content


In [13]:
! echo $PYTHONPATH
import os
os.environ['PYTHONPATH'] = "/env/python:/content/drive/MyDrive/00_work/004_pose_estimation/probreg/examples"
! echo $PYTHONPATH
import numpy as np
import open3d
def make_pcd(points, colors):
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(points)
    pcd.colors = open3d.utility.Vector3dVector(colors)
    return pcd
def make_merge(pcds):
  points=[]
  colors=[]
  for pcd in pcds:
     points.extend(np.array(pcd.points))
     colors.extend(np.array(pcd.colors))
  return make_pcd(points, colors)

/env/python:{bp}/contextdesc/
/env/python:/content/drive/MyDrive/00_work/004_pose_estimation/probreg/examples


##sample

In [15]:
import copy
import numpy as np
import open3d as o3
from probreg import cpd

# load source and target point cloud
bo='/content/drive/MyDrive/00_work/004_pose_estimation/probreg/result/'
# bp='/content/drive/MyDrive/00_work/004_pose_estimation/probreg/examples/'
bp='/content/drive/MyDrive/00_work/004_pose_estimation/contextdesc/imgs/'
# source = o3.io.read_point_cloud(f'{bp}bunny.pcd')
# target = copy.deepcopy(source)
# # transform target point 
# th = np.deg2rad(30.0)
# target.transform(np.array([[np.cos(th), -np.sin(th), 0.0, 0.0],
#                            [np.sin(th), np.cos(th), 0.0, 0.0],
#                            [0.0, 0.0, 1.0, 0.0],
#                            [0.0, 0.0, 0.0, 1.0]]))

source_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam0.ply')
target_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam1.ply')
# source = source_r.voxel_down_sample(voxel_size=0.005)
# target = target_r.voxel_down_sample(voxel_size=0.005)

source = source_r.voxel_down_sample(voxel_size=20)
target = target_r.voxel_down_sample(voxel_size=20)

# compute cpd registration
tf_param, _, _ = cpd.registration_cpd(source, target)
result = copy.deepcopy(source)
result.points = tf_param.transform(result.points)

# draw result
# source.paint_uniform_color([1, 0, 0])
# target.paint_uniform_color([0, 1, 0])
# result.paint_uniform_color([0, 0, 1])
ply_merge=make_merge([ target, result])

open3d.io.write_point_cloud(f'{bo}pcd_merge_probreg_cam01_cpd.ply', ply_merge)


True

##probreg/examples/cpd_affine3d_cuda.py 

##cpd_nonrigid3d_cuda.py cpd_affine3d_cuda.py

##function define

In [14]:
%cd /content/drive/MyDrive/00_work/004_pose_estimation/probreg/examples
import numpy as np
import copy
use_cuda = True
if use_cuda:
    import cupy as cp
    to_cpu = cp.asnumpy
    cp.cuda.set_allocator(cp.cuda.MemoryPool().malloc)
else:
    cp = np
    to_cpu = lambda x: x
import open3d as o3
import transformations as trans
from probreg import cpd
from probreg import callbacks
import utils
import time

#https://github.com/neka-nat/probreg/blob/master/examples/cpd_rigid_cuda.py
def reg_rigid_cpd(source,target):
  source_pt = cp.asarray(source.points, dtype=cp.float32)
  target_pt = cp.asarray(target.points, dtype=cp.float32)

  acpd = cpd.RigidCPD(source_pt, use_cuda=use_cuda)
  start = time.time()
  tf_param, _, _ = acpd.registration(target_pt)
  elapsed = time.time() - start
  print("time: ", elapsed)
  # print("result: ", to_cpu(tf_param.b), to_cpu(tf_param.t))

  points=tf_param.transform(source_pt)
  points_np = cp. asnumpy(points) 
  result=make_pcd(points_np,source.colors)
  return result,target
def reg_non_rigid_cpd(source,target):
  source_pt = cp.asarray(source.points, dtype=cp.float32)
  target_pt = cp.asarray(target.points, dtype=cp.float32)

  acpd = cpd.NonRigidCPD(source_pt, use_cuda=use_cuda)
  start = time.time()
  tf_param, _, _ = acpd.registration(target_pt)
  elapsed = time.time() - start
  print("time: ", elapsed)
  # print("result: ", to_cpu(tf_param.b), to_cpu(tf_param.t))

  points=tf_param.transform(source_pt)
  points_np = cp. asnumpy(points) 
  result=make_pcd(points_np,source.colors)
  return result,target
def reg_affine_cpd(source,target):
  source_pt = cp.asarray(source.points, dtype=cp.float32)
  target_pt = cp.asarray(target.points, dtype=cp.float32)

  acpd = cpd.AffineCPD(source_pt, use_cuda=use_cuda)
  start = time.time()
  tf_param, _, _ = acpd.registration(target_pt)
  elapsed = time.time() - start
  print("time: ", elapsed)
  # print("result: ", to_cpu(tf_param.b), to_cpu(tf_param.t))

  points=tf_param.transform(source_pt)
  points_np = cp. asnumpy(points) 
  result=make_pcd(points_np,source.colors)
  return result,target
def reg_cpd(source_pt,target_pt,fun_cpd):
  acpd = fun_cpd(source_pt, use_cuda=use_cuda)
  tf_param, _, _ = acpd.registration(target_pt)
  return tf_param
def tran_tf_param(tf_param,source_r):
  points_src=tf_param.transform(cp.asarray(source_r.points, dtype=cp.float32))
  points_src=to_cpu(points_src)
  result_trans=make_pcd(points_src,source_r.colors)
  return result_trans

/content/drive/MyDrive/00_work/004_pose_estimation/probreg/examples


In [ ]:
# print("result: ", to_cpu(tf_param.b), to_cpu(tf_param.t))

##RigidCPD

In [12]:
source_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam0.ply')
target_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam1.ply')

source = source_r.voxel_down_sample(voxel_size=20)
target = target_r.voxel_down_sample(voxel_size=20)
result,target_reg=reg_rigid_cpd(source,target)
ply_merge=make_merge([ target_reg, result])
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.RigidCPD.t.ply', target)
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.RigidCPD.s.ply', result)

time:  20.867454290390015


True

##AffineCPD

In [ ]:
source_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam0.ply')
target_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam1.ply')

source = source_r.voxel_down_sample(voxel_size=20)
target = target_r.voxel_down_sample(voxel_size=20)
result,target_reg=reg_affine_cpd(source,target)
ply_merge=make_merge([ target_reg, result])
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.AffineCPD.t.ply', target)
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.AffineCPD.s.ply', result)

##NonRigidCPD

In [ ]:
source_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam0.ply')
target_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam1.ply')

source = source_r.voxel_down_sample(voxel_size=20)
target = target_r.voxel_down_sample(voxel_size=20)
result,target_reg=reg_rigid_cpd(source,target)
ply_merge=make_merge([ target_reg, result])
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.NonRigidCPD.t.ply', target)
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.NonRigidCPD.s.ply', result)

## common call transform

In [10]:
bo='/content/drive/MyDrive/00_work/004_pose_estimation/probreg/result/'
bp='/content/drive/MyDrive/00_work/004_pose_estimation/contextdesc/imgs/'
source_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam0.ply')
target_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam1.ply')
source = source_r.voxel_down_sample(voxel_size=20)
target = target_r.voxel_down_sample(voxel_size=20)
source_pt = cp.asarray(source.points, dtype=cp.float32)
target_pt = cp.asarray(target.points, dtype=cp.float32)
# tf_param_non_rig=reg_cpd(source_pt,target_pt,cpd.NonRigidCPD)
tf_param_non_rig=reg_cpd(source_pt,target_pt,cpd.AffineCPD)
# tf_param_rig=reg_cpd(source_pt,target_pt,cpd.RigidCPD) cpd.AffineCPD
ply=tran_tf_param(tf_param_non_rig,source_r)
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg_AffineCPD_s2.ply', ply)

True

In [15]:
def generateTransMatrixRt(t, R):
    transMatrix = np.eye(4, 4)
    for y in range(0, 3):
        for x in range(0, 3):
            transMatrix[y, x] = R[y, x]
        transMatrix[y, 3] = t[y]
    return transMatrix

## test pts_obj1_aug

In [51]:
import open3d as o3
import cupy as cp
bo='/content/drive/MyDrive/00_work/004_pose_estimation/probreg/result/'
bp='/content/drive/MyDrive/00_work/004_pose_estimation/contextdesc/imgs/'
pts_obj1_aug_t,pts_obj2_aug_t=get_key_points(aug_match,cv_kpts_list[0],cv_kpts_list[1],imgdic)
source_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam0.ply')
target_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam1.ply')
source_pt = cp.asarray(pts_obj1_aug_t, dtype=cp.float32)
target_pt = cp.asarray(pts_obj2_aug_t, dtype=cp.float32)
tf_param_rig=reg_cpd(source_pt,target_pt,cpd.RigidCPD)
ply=tran_tf_param(tf_param_rig,source_r)
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.aug_match.RigidCPD.s.ply', ply)
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.aug_match.RigidCPD.t.ply', target_r)

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:160: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


True

##ransac_icp

In [18]:
#https://qiita.com/tttamaki/items/648422860869bbccc72d
import numpy as np
import open3d as py3d
import cv2
import os
# pip install open3d==0.10.0.1
# import pcl
class ransac_icp:
    def __init__(self):
        self.RANSAC = py3d.registration.registration_ransac_based_on_feature_matching
        self.ICP = py3d.registration.registration_icp
        self.FPFH = py3d.registration.compute_fpfh_feature
        self.GET_GTG = py3d.registration.get_information_matrix_from_point_clouds

    def register(self,pcd1, pcd2, size):
        # ペアの点群を位置合わせ

        kdt_n = py3d.geometry.KDTreeSearchParamHybrid(radius=size, max_nn=50)
        kdt_f = py3d.geometry.KDTreeSearchParamHybrid(radius=size *2, max_nn=50)

        # ダウンサンプリング
        pcd1_d = pcd1.voxel_down_sample( size)
        pcd2_d = pcd2.voxel_down_sample( size)
        pcd1_d.estimate_normals( kdt_n)
        pcd2_d.estimate_normals( kdt_n)
        print("voxel_down_sample:",np.array(pcd1_d.points).shape[0]/np.array(pcd1.points).shape[0])
        # 特徴量計算
        pcd1_f = self.FPFH(pcd1_d, kdt_f)
        pcd2_f = self.FPFH(pcd2_d, kdt_f)

        # 準備
        checker = [py3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                   py3d.registration.CorrespondenceCheckerBasedOnDistance(size * 2)]

        est_ptp = py3d.registration.TransformationEstimationPointToPoint()
        est_ptpln = py3d.registration.TransformationEstimationPointToPlane()

        criteria = py3d.registration.RANSACConvergenceCriteria(max_iteration=400000,
                                                  max_validation=500)
        # RANSACマッチング
        result1 = self.RANSAC(pcd1_d, pcd2_d,
                         pcd1_f, pcd2_f,
                         max_correspondence_distance=size * 2,
                         estimation_method=est_ptp,
                         ransac_n=4,
                         checkers=checker,
                         criteria=criteria)
        # ICPで微修正
        result2 = self.ICP(pcd1, pcd2, size, result1.transformation, est_ptpln)

        return result2.transformation


    def merge(self,pcds):
        # 複数の点群を1つの点群にマージする

        all_points = []
        all_colors = []
        for pcd in pcds:
            all_points.append(np.asarray(pcd.points))
            all_colors.append(np.asarray(pcd.colors))

        merged_pcd = py3d.geometry.PointCloud()
        merged_pcd.points = py3d.utility.Vector3dVector(np.vstack(all_points))
        merged_pcd.colors = py3d.utility.Vector3dVector(np.vstack(all_points))

        return merged_pcd


    def add_color_normal(self,pcd,size): # in-place coloring and adding normal
        # pcd.paint_uniform_color(np.random.rand(3))
        kdt_n = py3d.geometry.KDTreeSearchParamHybrid(radius=size, max_nn=50)
        pcd.estimate_normals( kdt_n)
        # py3d.estimate_normals(pcd,kdt_n)
        return pcd

    def load_pcds(self,pcd_files):

        pcds = []
        for f in pcd_files:
            pcd = py3d.io.read_point_cloud(f)
            pcd_tem=self.remove_noise_ply(pcd)
            cnt_org=np.array(pcd.points).shape[0]
            cnt_noise=np.array(pcd_tem.points).shape[0]
            if cnt_noise/cnt_org>0.5:
                pcd=pcd_tem
            pcds.append(pcd)
        return pcds


    def align_pcds(self,pcds, size):
        # 複数の点群を位置合わせ

        pose_graph = py3d.registration.PoseGraph()
        accum_pose = np.identity(4) # id0から各ノードへの累積姿勢
        pose_graph.nodes.append(py3d.registration.PoseGraphNode(accum_pose))

        n_pcds = len(pcds)
        for source_id in range(n_pcds):
            for target_id in range(source_id + 1, n_pcds):
                source = pcds[source_id]
                target = pcds[target_id]

                trans = self.register(source, target, size)
                GTG_mat = self.GET_GTG(source, target, size, trans) # これが点の情報を含む

                if target_id == source_id + 1: # 次のidの点群ならaccum_poseにposeを積算
                    accum_pose = trans @ accum_pose
                    pose_graph.nodes.append(py3d.registration.PoseGraphNode(np.linalg.inv(accum_pose))) # 各ノードは，このノードのidからid0への変換姿勢を持つので，invする
                    # そうでないならnodeは作らない
                pose_graph.edges.append(py3d.registration.PoseGraphEdge(source_id,
                                                           target_id,
                                                           trans,
                                                           GTG_mat,
                                                           uncertain=True)) # bunnyの場合，隣でも怪しいので全部True


        # 設定
        solver = py3d.registration.GlobalOptimizationLevenbergMarquardt()
        criteria = py3d.registration.GlobalOptimizationConvergenceCriteria()
        option = py3d.registration.GlobalOptimizationOption(
                 max_correspondence_distance=size / 10,
                 edge_prune_threshold=size / 10,
                 reference_node=0)

        # 最適化
        py3d.registration.global_optimization(pose_graph,
                                method=solver,
                                criteria=criteria,
                                option=option)

        # 推定した姿勢で点群を変換
        translst=[]
        for pcd_id in range(n_pcds):
            trans = pose_graph.nodes[pcd_id].pose
            print("transform matrix:",trans)
            pcds[pcd_id].transform(trans)
            translst.append(trans)


        return pcds,translst

    def make_pcd(self,points, colors):
      pcd = py3d.geometry.PointCloud()
      pcd.points = py3d.utility.Vector3dVector(points)
      pcd.colors = py3d.utility.Vector3dVector(colors)
      return pcd
    def merge(self,pcds):
        # 複数の点群を1つの点群にマージする

        all_points = []
        for pcd in pcds:
            all_points.append(np.asarray(pcd.points))

        merged_pcd = py3d.PointCloud()
        merged_pcd.points = py3d.Vector3dVector(np.vstack(all_points))

        return merged_pcd
    #https://github.com/aipiano/guided-filter-point-cloud-denoise/blob/master/main.py
    #https://github.com/sakizuki/SSII2018_Tutorial_Open3D/blob/master/Python/kdtree.py
    def guided_filter(self,pcd, radius=0.01, epsilon=0.1):
        kdtree = py3d.geometry.KDTreeFlann(pcd)
        points_copy = np.array(pcd.points)
        points = np.asarray(pcd.points)
        num_points = len(pcd.points)

        for i in range(num_points):
            # 1.RNNの方法
            k, idx, _ = kdtree.search_radius_vector_3d(pcd.points[i], radius)
            # 2.KNNの方法
            # k, idx, _ = kdtree.search_knn_vector_3d(pcd.points[i], 200)
            # 3.RKNNの方法
            # k, idx, _ = kdtree.search_hybrid_vector_3d(pcd.points[i], radius=0.1, max_nn=100)
            if k < 3:
                continue

            neighbors = points[idx, :]
            mean = np.mean(neighbors, 0)
            cov = np.cov(neighbors.T)
            e = np.linalg.inv(cov + epsilon * np.eye(3))

            A = cov @ e
            b = mean - A @ mean

            points_copy[i] = A @ points[i] + b

        pcd.points = py3d.utility.Vector3dVector(points_copy)
        # guided_filter(pcd, 0.01, 0.1)
        return pcd

    def remove_noise_ply(self,pcd):
        # (1) Load a ply point cloud, print it, and render it
        print("Load a ply point cloud, print it, and render it")
        # pcd = py3d.io.read_point_cloud(fn_pcd)
        # py3d.visualization.draw_geometries([pcd])
        # (2) Downsample the point cloud with a voxel
        print("Downsample the point cloud with a voxel of 0.02")
        voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.02)
        # py3d.visualization.draw_geometries([voxel_down_pcd])
        # (3) Every 5th points are selected
        print("Every 5th points are selected")
        uni_down_pcd = pcd.uniform_down_sample(every_k_points=5)
        # py3d.visualization.draw_geometries([uni_down_pcd])
        # (4) Statistical oulier removal
        print("Statistical oulier removal")
        cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=20,
                                                            std_ratio=2.0)
        # display_inlier_outlier(voxel_down_pcd, ind)
        # (5) Radius oulier removal
        print("Radius oulier removal")
        # cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=16, radius=0.05)
        cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=16, radius=2.5)
        inlier_cloud = voxel_down_pcd.select_by_index(ind)
        outlier_cloud = voxel_down_pcd.select_by_index(ind, invert=True)
        # inlier_cloud, outlier_cloud = display_inlier_outlier(voxel_down_pcd, ind)
        return inlier_cloud
import struct
def zed_depthfloat_to_abgr(f):
    """
    ZED pcd data format:
      ----------------------------------------------------------
      https://www.stereolabs.com/docs/depth-sensing/using-depth/
      ----------------------------------------------------------
      The point cloud stores its data on 4 channels using 32-bit 
      float for each channel. 
      The last float is used to store color information, where 
      R, G, B, and alpha channels (4 x 8-bit) are concatenated 
      into a single 32-bit float. 
  """
    # https://stackoverflow.com/questions/23624212/how-to-convert-a-float-into-hex/38879403
    if f == 0.:
        return [0, 0, 0, 0]
    else:
        h = hex(struct.unpack('<I', struct.pack('<f', f))[0])
        return [eval('0x' + a + b) for a, b in zip(h[::2], h[1::2]) if a + b != '0x']

##ransac_icp特徴ポイントの実施

In [19]:
pcd0,pcd1=get_key_ply(aug_match,cv_kpts_list[0],cv_kpts_list[1],imgdic)

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:160: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


In [13]:
ri=ransac_icp()
pcds=[pcd0,pcd1]
size=10
for pcd in pcds:
    si = np.abs((pcd.get_max_bound() - pcd.get_min_bound())).max() / 40
    if  size < si:
        size=si
print("size:",size)
pcdsn=[]
for pcd in pcds:
    pcdn=ri.add_color_normal(pcd,size)
    pcdsn.append(pcdn)

pcd_aligned,translst = ri.align_pcds(pcdsn, size)

size: 469.05395660400393
voxel_down_sample: 0.23235887096774194
[Open3D WARNING] Certain-edge subset of PoseGraph is not connected.
transform matrix: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
transform matrix: [[ 9.70911699e-01  7.04301323e-02 -2.28845076e-01  5.56762669e+02]
 [-8.12062099e-02  9.95972420e-01 -3.80064543e-02  7.24299902e+01]
 [ 2.25246585e-01  5.54845524e-02  9.72720638e-01  8.64661677e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [24]:
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01.features.ransac_icp_0.ply', pcd_aligned[0])
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01.features.ransac_icp_1.ply', pcd_aligned[1])

True

In [23]:
pcd_aligned

[geometry::PointCloud with 1984 points.,
 geometry::PointCloud with 1984 points.]

In [18]:
source_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam0.ply')
target_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam1.ply')
target_trans=target_r.transform(translst[1])

In [20]:

bo='/content/drive/MyDrive/00_work/004_pose_estimation/probreg/result/'
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.ransac_icp.t.ply', target_trans)
open3d.io.write_point_cloud(f'{bo}pcd_merge_cam01_probreg.ransac_icp.s.ply', source_r)

True

## time_measurement.py

In [22]:
# %cd /content/drive/MyDrive/00_work/004_pose_estimation/probreg/examples
import os
os.environ['PYTHONPATH'] = "/env/python:/content/drive/MyDrive/00_work/004_pose_estimation/probreg/examples"
from timeit import default_timer as timer
import numpy as np
import open3d as o3
import utils
from probreg import cpd
from probreg import l2dist_regs
from probreg import gmmtree
from probreg import filterreg
import open3d
threshold = 0.001
max_iteration = 100

In [25]:
bo='/content/drive/MyDrive/00_work/004_pose_estimation/probreg/result/'
bp='/content/drive/MyDrive/00_work/004_pose_estimation/contextdesc/imgs/'
source_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam0.ply')
target_r = o3.io.read_point_cloud(f'{bp}pcd_mask_cam1.ply')
source,target=get_key_ply(aug_match,cv_kpts_list[0],cv_kpts_list[1],imgdic)
source_pt = cp.asarray(source.points, dtype=cp.float32)
target_pt = cp.asarray(target.points, dtype=cp.float32)

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:160: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


In [26]:
res_icp = o3.registration.registration_icp(source, target, 20,
                                       np.identity(4), o3.registration.TransformationEstimationPointToPoint(),
                                       o3.registration.ICPConvergenceCriteria(max_iteration=max_iteration))
ply=source_r.transform(res_icp.transformation)
open3d.io.write_point_cloud(f'{bo}feat_pcd_merge_cam01_probreg.aug_match.s.icp.ply', ply)

True

In [27]:
res_cpd = cpd.registration_cpd(source, target, maxiter=max_iteration, tol=threshold)
r=res_cpd.transformation.rot
t=res_cpd.transformation.t
trans=generateTransMatrixRt(t,r)
ply=source_r.transform(res_icp.transformation)
open3d.io.write_point_cloud(f'{bo}feat_pcd_merge_cam01_probreg.aug_match.s.cpd.ply', ply)

True

In [32]:
res_svr = l2dist_regs.registration_svr(source, target, opt_maxiter=max_iteration, opt_tol=threshold)
r=res_svr.rot
t=res_svr.t
trans=generateTransMatrixRt(t,r)
ply=source_r.transform(res_icp.transformation)
open3d.io.write_point_cloud(f'{bo}feat_pcd_merge_cam01_probreg.aug_match.s.svr.ply', ply)

True

In [29]:
res_gmmtree = gmmtree.registration_gmmtree(source, target, maxiter=max_iteration, tol=threshold)
r=res_gmmtree.transformation.rot
t=res_gmmtree.transformation.t
trans=generateTransMatrixRt(t,r)
ply=source_r.transform(res_icp.transformation)
open3d.io.write_point_cloud(f'{bo}feat_pcd_merge_cam01_probreg.aug_match.s.gmmtree.ply', ply)

True

In [30]:
res_filterreg = filterreg.registration_filterreg(source, target,
                                       sigma2=None, maxiter=max_iteration, tol=threshold)
r=res_filterreg.transformation.rot
t=res_filterreg.transformation.t
trans=generateTransMatrixRt(t,r)
ply=source_r.transform(res_icp.transformation)
open3d.io.write_point_cloud(f'{bo}feat_pcd_merge_cam01_probreg.aug_match.s..filterreg.ply', ply)

True

True

In [ ]:
# !pip install -r requirements.txt

In [ ]:
# %cd /content/lf-net-release
# !python run_lfnet.py \
#   --in_dir=/content/lf-net-release/samples \
#   --out_dir=/content/lf-net-release/dump_feats

In [ ]:
# %cd /content/lf-net-release/models
# !pwd

In [ ]:
# img_name = 'london_bridge-00001.jpg'
# img_path = '../samples/'+img_name
# kp_path = '../dump_feats/{}.npz'.format(img_name)
# image = imread(img_path).astype(np.float32) / 255
# kp_info = np.load(kp_path)
# kpts = kp_info['kpts']
# feats = kp_info['descs']
# height, width = kp_info['size']
# image = cv2.resize(image, (width, height))

# print('Load {}-kpts, feat={}-D imsize={}x{}'.format(kpts.shape[0], feats.shape[1], width, height))

In [ ]:
# def get_kpt_desc(img_name):
#   # img_name = 'london_bridge-00001.jpg'
#   img_path = '../samples/'+img_name
#   kp_path = '../dump_feats/{}.npz'.format(img_name)
#   image = imread(img_path).astype(np.float32) / 255
#   kp_info = np.load(kp_path)
#   kpts = kp_info['kpts']
#   feats = kp_info['descs']
#   height, width = kp_info['size']
#   image = cv2.resize(image, (width, height))
#   return kpts,feats,image


In [ ]:
# def get_good_match_knn(bf,desc1,desc2):
#     # マッチング器を作成する。
#     matches = bf.knnMatch(desc1, desc2, k=2)
#     # レシオテストを行う。
#     good_matches = []
#     thresh = 0.3
#     for first, second in matches:
#         if first.distance < second.distance * thresh:
#             good_matches.append(first)
#     print("good_matches:",len(good_matches))
#     return good_matches
# kpts1,desc1,img1= get_kpt_desc('london_bridge-00001.jpg')
# kpts2,desc2,img2=get_kpt_desc('london_bridge-00002.jpg')
# bf = cv2.BFMatcher(cv2.NORM_HAMMING)
# good_matches=get_good_match(bf,desc1,desc2)